<a href="https://colab.research.google.com/github/Mustansir-52/Alpha/blob/main/Movie_Recommendation_System_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

class MovieRecommender:
    def __init__(self):
        # 1. THE DATASET
        # A built-in dataset for immediate testing.
        self.data = [
            {
                "title": "The Dark Knight",
                "genre": "Action Crime Drama",
                "description": "When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice."
            },
            {
                "title": "Inception",
                "genre": "Action Adventure Sci-Fi",
                "description": "A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into the mind of a C.E.O."
            },
            {
                "title": "Interstellar",
                "genre": "Adventure Drama Sci-Fi",
                "description": "A team of explorers travel through a wormhole in space in an attempt to ensure humanity's survival."
            },
            {
                "title": "Toy Story",
                "genre": "Animation Adventure Comedy",
                "description": "A cowboy doll is profoundly threatened and jealous when a new spaceman figure supplants him as top toy in a boy's room."
            },
            {
                "title": "The Godfather",
                "genre": "Crime Drama",
                "description": "The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son."
            },
            {
                "title": "Pulp Fiction",
                "genre": "Crime Drama",
                "description": "The lives of two mob hitmen, a boxer, a gangster and his wife, and a pair of diner bandits intertwine in four tales of violence and redemption."
            },
            {
                "title": "The Matrix",
                "genre": "Action Sci-Fi",
                "description": "A computer hacker learns from mysterious rebels about the true nature of his reality and his role in the war against its controllers."
            },
            {
                "title": "Finding Nemo",
                "genre": "Animation Adventure",
                "description": "After his son is captured in the Great Barrier Reef and taken to Sydney, a timid clownfish sets out on a journey to bring him home."
            },
            {
                "title": "Goodfellas",
                "genre": "Biography Crime Drama",
                "description": "The story of Henry Hill and his life in the mob, covering his relationship with his wife Karen Hill and his mob partners Jimmy Conway and Tommy DeVito."
            },
            {
                "title": "Star Wars: Episode IV - A New Hope",
                "genre": "Action Adventure Fantasy",
                "description": "Luke Skywalker joins forces with a Jedi Knight, a cocky pilot, a Wookiee and two droids to save the galaxy from the Empire's world-destroying battle station."
            },
            {
                "title": "Avengers: Endgame",
                "genre": "Action Adventure Drama",
                "description": "After the devastating events of Infinity War, the universe is in ruins. With the help of remaining allies, the Avengers assemble once more in order to reverse Thanos' actions."
            },
             {
                "title": "Coco",
                "genre": "Animation Adventure Family",
                "description": "Aspiring musician Miguel, confronted with his family's ancestral ban on music, enters the Land of the Dead to find his great-great-grandfather, a legendary singer."
            }
        ]

        self.df = pd.DataFrame(self.data)

        # 2. FEATURE ENGINEERING
        self.df['content'] = self.df['genre'] + " " + self.df['description']

        # 3. TRAIN MODEL
        self._train_model()

    def _train_model(self):
        # Create TF-IDF Matrix
        tfidf = TfidfVectorizer(stop_words='english')
        self.tfidf_matrix = tfidf.fit_transform(self.df['content'])

        # Compute Cosine Similarity
        self.cosine_sim = linear_kernel(self.tfidf_matrix, self.tfidf_matrix)

        # Map titles to indices
        self.indices = pd.Series(self.df.index, index=self.df['title'].str.lower()).drop_duplicates()

    def get_recommendations(self, title, num_recommendations=3):
        try:
            idx = self.indices[title.lower()]
            if isinstance(idx, pd.Series):
                idx = idx.iloc[0]

            sim_scores = list(enumerate(self.cosine_sim[idx]))
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
            sim_scores = sim_scores[1:num_recommendations+1]
            movie_indices = [i[0] for i in sim_scores]
            return self.df.iloc[movie_indices][['title', 'genre', 'description']]

        except KeyError:
            return None

    def display_menu(self):
        print("\n" + "="*50)
        print(" MOVIE SELECTION MENU")
        print("="*50)
        print(f"{'ID':<5} | {'Title':<40} | {'Genre'}")
        print("-" * 50)
        for index, row in self.df.iterrows():
            # Adding 1 to index so the menu starts at 1 instead of 0
            print(f"{index + 1:<5} | {row['title']:<40} | {row['genre']}")
        print("="*50)

def main():
    app = MovieRecommender()

    while True:
        # 1. Show the menu to the user
        app.display_menu()

        # 2. Let the user decide by typing a number
        user_input = input("\nEnter the Movie ID to select (or 'q' to quit): ").strip()

        if user_input.lower() == 'q':
            print("Goodbye!")
            break

        # 3. Validate Input
        if not user_input.isdigit():
            print("❌ Invalid input. Please enter a number.")
            continue

        selected_id = int(user_input) - 1 # Convert back to 0-based index

        # Check if ID is valid
        if 0 <= selected_id < len(app.df):
            selected_movie = app.df.iloc[selected_id]
            title = selected_movie['title']

            print(f"\n✅ You selected: {title}")
            print(f"   Plot: {selected_movie['description']}")

            # 4. Get Recommendations
            recommendations = app.get_recommendations(title)

            print(f"\n🎥 Recommended Movies based on '{title}':")
            for i, row in recommendations.iterrows():
                print(f"   • {row['title']} ({row['genre']})")

            input("\nPress Enter to choose another movie...")
        else:
            print(f"❌ ID {user_input} is out of range. Please select a number from the list.")

if __name__ == "__main__":
    main()


 MOVIE SELECTION MENU
ID    | Title                                    | Genre
--------------------------------------------------
1     | The Dark Knight                          | Action Crime Drama
2     | Inception                                | Action Adventure Sci-Fi
3     | Interstellar                             | Adventure Drama Sci-Fi
4     | Toy Story                                | Animation Adventure Comedy
5     | The Godfather                            | Crime Drama
6     | Pulp Fiction                             | Crime Drama
7     | The Matrix                               | Action Sci-Fi
8     | Finding Nemo                             | Animation Adventure
9     | Goodfellas                               | Biography Crime Drama
10    | Star Wars: Episode IV - A New Hope       | Action Adventure Fantasy
11    | Avengers: Endgame                        | Action Adventure Drama
12    | Coco                                     | Animation Adventure Family

Enter th

KeyboardInterrupt: Interrupted by user